In [2]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [3]:
root = './NewCleanDataSetBRCA/'
rna_data = sio.loadmat(root+'Data_RNASeq2.mat')

In [4]:
cancerfreedata = rna_data['geneRNASeq2RawMatrix0']
cancerobtdata = rna_data['geneRNASeq2RawMatrix1']

In [4]:
target0 = np.zeros((1, cancerfreedata.shape[1]))
target1 = np.ones((1, cancerobtdata.shape[1]))

In [5]:
total_data = np.concatenate([cancerfreedata, cancerobtdata], axis=1)

In [14]:
print('number of genome is', total_data.shape[0])

number of genome is 20531


In [7]:
graph_info = sio.loadmat(root+'link.mat')

In [8]:
edge_index = graph_info['EdgeConn'][:,:2]

In [9]:
print('The total number of node in linked dataset is ', np.unique(edge_index[:]).shape)

The total number of node in linked dataset is  (12129,)


In [10]:
# Here we should map the two dataset of genome
RNAname_list = rna_data['gNamePureListRNASeq2'][0]

In [11]:
PPIname_list = sio.loadmat(root+'nodename.mat')['irefGenes'][0]

In [21]:
ret = np.isin(RNAname_list, PPIname_list)

In [24]:
datainfo = total_data[ret]

In [25]:
datainfo.shape

(11864, 1190)

In [28]:
avail_PPIdata = RNAname_list[ret]

In [29]:
avail_PPIdata.shape

(11864,)

In [34]:
# 对于原来12129个中找出 实际有效的T/F
ret2 = np.isin(PPIname_list, avail_PPIdata, assume_unique=True)

In [73]:
## Start to deal with the edge info

In [55]:
# 通过名字得到的有效genome, 这里有11864 个
avail_index = np.array(range(1, ret2.shape[0]+1))[ret2]

In [61]:
# 这里能找到实际名字在rna数据中涵盖的link信息
temp = np.isin(edge_index, avail_index)

In [84]:
res = temp[:,0] & temp[:,1]

In [70]:
#筛选过后 linkde大小为89334
edge_avail = edge_index[res]

In [125]:
res2 = np.isin(avail_index, np.unique(edge_avail.reshape(-1)))

In [128]:
avail_PPIdata = avail_PPIdata[res2]

In [130]:
datainfo = datainfo[res2]

In [132]:
avail_index = avail_index[res2]

In [138]:
#第二遍筛选，剔除孤立点
temp = np.isin(edge_avail, avail_index)
res = temp[:,0] & temp[:,1]
edge_avail = edge_avail[res]

In [151]:
edge_avail_self = np.concatenate([edge_avail, np.array([avail_index]*2).T], axis=0)
edge_avail_self = np.unique(edge_avail_self, axis=0)

In [131]:
root2 = './processed-rna-genome-data/'
pd.DataFrame(avail_PPIdata).to_csv(root2+'availPPidata_name.csv',index=False)
pd.DataFrame(datainfo).to_csv(root2+'availPPidata.csv',index=False)

In [140]:
pd.DataFrame(edge_avail_self).to_csv(root2+'availedge.csv',index=False)

In [153]:
le = preprocessing.LabelEncoder()
le.fit(edge_avail_self.reshape(-1))
availedge_index =le.transform(edge_avail_self.reshape(-1)).reshape(-1,2)

In [154]:
availedge_index

array([[    0,     0],
       [    0,  2338],
       [    0,  5578],
       ...,
       [11829, 11829],
       [11830, 11830],
       [11831, 11831]])

In [155]:
edge_avail_self

array([[    1,     1],
       [    1,  2397],
       [    1,  5707],
       ...,
       [12127, 12127],
       [12128, 12128],
       [12129, 12129]])

In [156]:
availedge_index.max()

11831

In [157]:
edge_avail_self.max()

12129